In [4]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

### Data preparationdataset

In [14]:
import cicids2017
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.manifold import TSNE
from ProphetStorm import ProphetStorm
import datetime as dt

RES_PATH = "."
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")


dataset = cicids2017.load_dataset(DATASETPATH_CACHE)
tr = dataset["TR"]
# tr = [ x.sample(frac=.01) for x in tr ]

ps = ProphetStorm()
ps.fit(tr)

100%|██████████| 14/14 [1:13:32<00:00, 315.16s/it]


### Training prophet storm

In [18]:
dt = [dataset["DT"]]

netdata = ps.isanomaly(dt)
Y = pd.concat(dt)["_isanomaly"]
Y = (Y=="none").replace({True: 0, False: 1}).values
y_hat = netdata["_y_hat"].values

100%|██████████| 14/14 [02:24<00:00, 10.33s/it]


In [19]:
from sklearn import metrics

# report = metrics.classification_report(y_attacks, y_hat)
metrics_rep = [ metrics.roc_auc_score,
                metrics.precision_score, metrics.recall_score,
                metrics.accuracy_score, metrics.f1_score]
for m in metrics_rep:
    mres = m(Y, y_hat)
    print(f"{m.__name__}(moday+attacks): {mres}")

tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
print("\n Confusion matrix")
print(f"\ttp: {tp} \tfp: {fp} \n\tfn: {fn} \ttn: {tn}")
# print(f"\n{report}")

roc_auc_score(moday+attacks): 0.5
precision_score(moday+attacks): 0.15090460526315788
recall_score(moday+attacks): 1.0
accuracy_score(moday+attacks): 0.15090460526315788
f1_score(moday+attacks): 0.26223651304037154

 Confusion matrix
	tp: 0.15090460526315788 	fp: 0.8490953947368421 
	fn: 0.0 	tn: 0.0
